<a href="https://colab.research.google.com/github/ShahmeerShahid/KeywordFinder-NLP_Model/blob/master/Key_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
df=pd.read_csv("/content/drive/My Drive/Uni/monster_com-job_sample.csv")

In [0]:
df.describe()

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
count,22000,22000,122,22000,22000,22000,22000,20372,22000,15133,22000,3446,16806,22000
unique,1,1,78,1,1,18744,18759,39,8423,738,22000,1737,163,22000
top,United States of America,US,9/22/2016,No,jobs.monster.com,12N Horizontal Construction Engineers Job Desc...,Monster,Full Time,"Dallas, TX",Healthcare Services,http://jobview.monster.com/security-officer-jo...,"40,000.00 - 100,000.00 $ /year",Experienced (Non-Manager),49938d31cd9c7c425eaeaec083db4da5
freq,22000,22000,6,22000,22000,104,318,6757,646,1919,1,50,4594,1


In [0]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def lower_and_remove_nums(text):
  text = text.lower() # convert to lower case
  text = re.sub("(\\d|\\W|\s+)", " ", text).strip() # remove digits, special characters,
                                                    # and consecutive spaces
  return text
def lemmatize(text):
  # lemmatization is the process of reducing words to their base form in order to
  # normalize them
  lemmatizer = WordNetLemmatizer()
  lemmatized = [lemmatizer.lemmatize(tokenized) for tokenized in word_tokenize(text)]
  return " ".join(lemmatized)
def pre_process(text):
  return lemmatize(lower_and_remove_nums(text))

# We can combine both the job titles and job descriptions
df['text'] = df['job_title'].apply(lambda x: x + " Canada ") + df['job_description']
df = df[['text']]
df['text'] = df['text'].apply(lambda x: pre_process(x))

In [0]:
def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

stopwords=get_stop_words("/content/drive/My Drive/Uni/stopwords.txt")

In [0]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=20):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [0]:
# An example of a pre-processed job description
from pprint import pprint
pprint(df['text'][1])

('Business Reporter/Editor Job in Madison Canada The Wisconsin State Journal '
 'is seeking a flexible and motivated reporter/editor to lead its three-person '
 'business desk. We’re looking for an experienced journalist able to spot '
 'trends, be a watchdog and reflect the Madison area’s vibrant entrepreneurial '
 'community. This is a hybrid reporting and editing position that calls for a '
 'fleet-footed, multimedia storyteller able to maximize the newspaper’s online '
 'presentation while also editing two sections a week. Candidates must have '
 'strong news judgment, be well versed in business news and trends and be able '
 'to quickly prioritize coverage.At least five years’ experience reporting or '
 'editing for digital and print platforms desired. To be considered for the '
 'position, applicants must apply online.\xa0As part of your online '
 'application, please attach five samples of your work or links to five recent '
 'stories.Wisconsin State Journal, 1901 Fish Hatchery 

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
docs = df['text'].tolist()
cv = CountVectorizer(max_df=0.85, stop_words=stopwords, max_features=10000)
word_count_vector = cv.fit_transform(docs)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [0]:
# We can look at the top 10 words in our vocabulary
list(cv.vocabulary_.keys())[:10]

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [0]:
# We can now test the transformer on a real job posting
# Below is an example job posting from Yelp found on Indeed.ca
test_post = """Software Engineer - Backend Infrastructure (Toronto) Yelp’s mission is to connect users with great local businesses. We are an agile team that creates an amazing experience for millions of people and business owners. We are growing our engineering teams to build, experiment and iterate on new product offerings and improve user experience. One of our challenges is to assist our high velocity engineering teams with speedy delivery of quality products to our users.

We are looking for tenacious and driven engineers to join our Engineering Effectiveness teams. Our mission is to help developers ship products quickly, reliably and securely and to provide them with the infrastructure they need to build robust and scalable features for our millions of users. We’ve evolved our service oriented architecture, developed API infrastructure for our mobile apps, built scalable CI/CD platforms and continue to do more. We also build tools to analyze tests and surface effectiveness health metrics for all engineering teams at Yelp. We support hundreds of microservices, thousands of docker containers and hundreds of developers.

As an engineer in Engineering Effectiveness, you will be working closely with the rest of engineering to take on problems of scale and efficiency. Yelp engineering culture is driven by our values: we’re a cooperative team that values individual authenticity, and encourages “unboring” solutions to problems.
What You Will Do:

    Design, build and deploy scalable and reliable infrastructure for backend services and APIs.
    Support and improve our tools for continuous build, automated testing, source control, and release management.
    Rapidly deliver and maintain high-performance, scalable systems, tools and processes to meet the needs of our rapidly growing engineering teams.
    Work closely with other engineers within your group and across the engineering organization.

What We Are Looking For:

    Experience designing scalable infrastructure and backend systems, with an understanding of operational and reliability trade-offs.
    Familiarity with containerization technologies such as Docker, Kubernetes etc. and knowledge of distributed systems.
    Proficient in one of Python, Java, C++, or Go.
    Excellent interpersonal and communication skills.
    3+ years of relevant industry experience, if you have less than 1 year of experience please consider applying for our New Grad positions.
    We are open to remote employees for this role.


Yelp values diversity. We’re proud to be an equal opportunity employer and consider qualified applicants without regard to race, color, religion, sex, national origin, ancestry, age, genetic information, sexual orientation, gender identity, marital or family status, veteran status, medical condition or disability.

Note: Yelp does not accept agency resumes. Please do not forward resumes to any recruiting alias or employee. Yelp is not responsible for any fees related to unsolicited resumes."""


In [0]:
feature_names = cv.get_feature_names()
tf_idf_vector = tfidf_transformer.transform(cv.transform([test_post]))
sorted_items = sort_coo(tf_idf_vector.tocoo())
keywords = extract_topn_from_vector(feature_names, sorted_items, 10)
print(keywords)

{'scalable': 0.4, 'engineering': 0.347, 'infrastructure': 0.274, 'backend': 0.27, 'docker': 0.212, 'build': 0.195, 'effectiveness': 0.176, 'rapidly': 0.122, 'securely': 0.102, 'velocity': 0.101}
